#1-5 scikit-learnの利用

##概要

scikit-learnは、Pythonプログラマにとって、一般的な機械学習ツールのトップに立つものになりました。
scikit-learnには、次のようなサブモジュールがあります。

*   分類：SVMや最近傍法、ランダムフォレスト、ロジスティック回帰など
*   回帰：Lasso、リッジ回帰、など
*   クラスタリング：k 平均、スペクトラルクラスタリング、など
*   次元削減：PCA、特徴量選択、マトリクスファクトリゼーション（Matrix Factorization）、など
*   モデル選択：グリッドサーチ（Grid Search）、交差検証（Cross Validation）、メトリクス
*   前処理：特徴量抽出、正規化

##iris(あやめ)のデータの例

###データの取り込み

この例で用いるデータは、機械学習や統計で古くから用いられているirisデータセットであります。
これは、scikit-learnのdatasetsモジュールに含まれていて、load_iris関数で読み込むことができます。

<img src="https://github.com/t-date/DataScience/blob/master/fig/01_04_04_01.jpg?raw=true" width="180px">

In [0]:
from sklearn.datasets import load_iris
iris_dataset = load_iris()
print("Keys of iris_dataset: \n{}".format(iris_dataset.keys()))

キーDESCRの値は、データセットの簡単な説明（description）です。ここでは説明の最初の部分だけ見てみしょう。

In [0]:
print(iris_dataset['DESCR'][:193]+ "\n...")

In [0]:
print("Target names: {}".format(iris_dataset['target_names']))

キーfeature_namesに対応する値は文字列のリストで、それぞれの特徴量の説明が格納されています。

In [0]:
print("Feature names: \n{}".format(iris_dataset['feature_names']))

データ本体は、targetとdataフィールドに格納されている。dataには、ガクの長さ、ガクの幅、花弁の長さ、花弁の幅がNumPy配列として格納されています。

In [0]:
print("Type of data: {}".format(type(iris_dataset['data'])))

配列dataの行は個々の花に対応し、列は個々の花に対して行われた4つの測定に対応します。

In [0]:
print("Shape of data: {}".format(iris_dataset['data'].shape))

配列には150の花の測定結果が格納されている。機械学習では、個々のアイテムをサンプルと呼
び、その特性を特徴量と呼びます。data配列のshapeはサンプルの個数かける特徴量
の数であります。これはscikit-learnで慣例として用いられている表現で、常にこの形になっています。
最初の5つのサンプルを見てみましょう。

In [0]:
print("First five columns of data:\n{}".format(iris_dataset['data'][:5]))

このデータから、最初の5つの花は花弁の幅がすべて0.2cmで、5つの中では最初の花が最も長い5.1cmのガクを持っていることがわかります。
配列targetには、測定された個々の花の種類が、やはりNumPy配列として格納されています。

In [0]:
print("Type of target: {}".format(type(iris_dataset['target'])))

targetは1次元の配列で、個々の花に1つのエントリが対応します。

In [0]:
print("Shape of target: {}".format(iris_dataset['target'].shape))

種類は0から2までの整数としてエンコードされています。

In [0]:
print("Target:\n{}".format(iris_dataset['target']))

これらの数値の意味は、配列iris_dataset['target_names']で与えられる。0はsetosaを、1はversicolorを、2はvirginicaを意味します。

###訓練データとテストデータ

新たに計測したアイリスに対してその種類を予測する機械学習モデルを構築しようとする場合、構築したモデルを新たに計測したデータに適用する前に、そのモデルが実際に機能するのか、つまり予測を信じてよいのかどうかを知っておく必要があります。

残念ながら、モデルを構築するのに使ったデータを、モデルの評価に使うことはできません。これは、モデルが単純に訓練データをまるまる覚えてしまい、訓練データに含まれているどのデータポイントに対しても常に正確にラベルを予測できるようになってしまうからです。このように「覚えて」いるだけでは、モデルがうまく汎化（generalize）できている（つまり新たなデータに対してもうまく機能する）ことの指標にはなりません。

モデルの性能を評価するには、ラベルを持つ新しいデータ（これまでに見せていないデータ）を使う必要があり、これを実現するには、集めたラベル付きデータ（ここでは150のアイリスの測定結果）を2つに分けるのが一般的であります。

一方のデータを機械学習モデルの構築に用います。こちらを訓練
データ（training data）もしくは訓練セット（training set）と呼びます。残りのデータを使ってモデルがどの程度うまく機能するかを評価する。こちらはテストデータ（test data）、テストセット（test set）もしくはホールドアウトセット（hold-out set）と呼ばれます。
scikit-learnには、データセットを並べ替えて、分割するtrain_test_splitという関数が用意されています。この関数はデータとラベルの75%を取り出して訓練セットにし、残り25%をテストセットにします。訓練セットとテストセットに割り当てるデータの量には厳密な決まりはないが、目安としてはテストに25%程度を割り当てるのが一般的です。

scikit-learnでは、データを大文字のXで、ラベルを小文字のyで示すのが一般的です。これは数学での標準的な数式の書き方f (x)＝y から来ています。ここではx が関数への入力でy が出力であります。
さらに数学での慣例に従い、2次元配列（行列）であるデータには大文字Xを、1次元配列（ベクトル）であるラベルには小文字のyを用います。
train_test_split関数を呼び出した結果を、この命名規則に従った変数に代入してみましょう。

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
iris_dataset['data'], iris_dataset['target'], random_state=0)

train_test_split関数は、分割を行う前に擬似乱数を用いてデータセットをシャッフルします。
データポイントはラベルでソートされているので、単純に最後の25%をテストセットにすると、すべてのデータポイントがラベル2になってしまいます。3クラスのうち1つしか含まれていないようなデータセットでは、モデルの汎化がうまくいっているか判断できません。そのため、先にデータをシャッフルし、テストデータにすべてのクラスが含まれるようにします。

同じ関数を何度か呼び出した際に、確実に同じ結果が得られるよう、random_stateパラメータを用いて擬似乱数生成器に同じシードを渡しています。これによって出力が決定的になり、常に同じ結果が得られるようになります。本書では、乱数を用いる際には常にこのようにrandom_stateパラメータを固定して用いています。
関数train_test_splitの出力はX_train、X_test、y_train、y_testとなる。これらはすべてNumPy配列で、X_trainにはデータセットの75%の行が、X_testには残りの25%の行が含まれています。

In [0]:
print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))

In [0]:
print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))

###データの観察

機械学習モデルを構築する前に、データを検査したほうがよいです。機械学習を用いなくても簡単に解ける問題かもしれないし、データに必要な情報が含まれていないかもしれないからです。

さらに、データを検査することで、データ内の異常値やおかしな点を見つけることができます。例えば、アイリスの花の一部がセンチメートルでなくインチで計測されているかもしれません。実際の世界では、データが不整合だったり、測定がおかしかったりすることは珍しくありません。

データを検査する最良の方法は、可視化です。その方法の1つが散布図です。散布図とは、x 軸にある特徴量を、y 軸にもう1つの特徴量を取り、データポイントごとにドットをプロットするものです。残念ながら、計算機のスクリーンは2次元なので、2つ（ぎりぎり3つ）までの特徴量しか同時にはプロットできません。したがって、3つを超える特徴量を持つデータセットをプロットすることは難しいです。この問題を回避する方法の1つがペアプロットで、すべての組合せ可能な特徴量の組合せをプロットするものです。特徴量の数が少ない場合（ここでは4つ）にはこの方法はうまくいきます。しかし、ペアプロットではすべての特徴量の相関を同時に見ることはできないため、この方法で可視化し
てもデータの興味深い側面を見ることができない場合があることに注意しましょう。

下に訓練セットの特徴量のペアプロットを示す。データポイントの色はアイリスの種類を示す。このグラフを作成するには、まずNumPy配列をpandasのDataFrameに変換します。pandasはscatter_matrixと呼ばれるペアプロットを作成する関数を持つ。グラフマトリックスの対角部分に
は、個々の特徴量のヒストグラムが描画されます。

In [0]:
#スクラッチの描画ライブラリをインポート
!pip install mglearn

In [0]:
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import mglearn

# X_trainのデータからDataFrameを作る、
# iris_dataset.feature_namesの文字列を使ってカラムに名前を付ける。
iris_dataframe = pd.DataFrame(X_train, columns=iris_dataset.feature_names)
# データフレームからscatter matrixを作成し、y_trainに従って色を付ける。
grr = pd.plotting.scatter_matrix(iris_dataframe, c=y_train, figsize=(8, 8), marker='o',
                                  hist_kwds={'bins': 20}, s=60, alpha=.8,cmap=mglearn.cm3)

3つのクラスは花弁とガクの測定結果で比較的よく分離していることがわかります。これは、うまく分離できるように機械学習モデルを訓練することができる可能性が高いことを意味します。